## Сравнение методов вычисления потокосцепления

Блокнот создан для подтверждения, что ноимнальный режим СДПМ преставляет собой режим работы без обратной связи, при котором на двигатель просто подается номинальное напряжение с номинальной частотой, а ротор двигателя нагружается номинальной нагрузкой. Далее этот режим называется частотный.

Проверяются следующие альтернативы
* параметеры номинального режима снимаются в режиме частотного управления (ветвь 1 решения квадратного уравнения),
* параметеры номинального режима снимаются в режиме частотного управления (ветвь 2 решения квадратного уравнения),
* параметеры номинального режима снимаются в режиме полеориентированного управления (MTPA),
* параметеры номинального режима снимаются в режиме BLDC (напряжение всегда подается вдоль квадратурной оси q).

### Модель вывода парметров СДПМ без вычисления/проверки значения потокосцепления

Нужна, чтобы не возникало исключений, из-за несгласованности номинального режима и занячения потокосцепления.

In [ ]:
import numpy as np
from deductor import DeductorBaseNamed, BaseAttribute, DerivedAttribute, AliasAttribute, ScaledAliasAttribute, DerivateRule, Validator
from pmsm import ModelRotary

# model with nominal mode without flux calculation
class MotorPlateParametersRotary(ModelRotary): 
    _ATTRIBUTES = [
        BaseAttribute('Un', 'V', 'Rated phase voltage amplitude (beeween phase and zero point).'),
        BaseAttribute('In', 'A', 'Rated current amplitude (on phase line).'),
        BaseAttribute('vn', 'rad/s', 'Rated speed.'),
        BaseAttribute('Tn', 'Nm', 'Rated torque.'),
        AliasAttribute('rated_speed', 'vn', groups=['rated_speed','rated']),
        AliasAttribute('rated_torque', 'Tn', groups=['rated_torque','rated']),
        ScaledAliasAttribute('rated_ac_voltage', np.sqrt(3/2), 'Un', 'V', 'Rated 3-phase rms voltage', groups=['rated_voltage', 'rated']),
        ScaledAliasAttribute('Un_rms', np.sqrt(1/2), 'Un', 'V', 'Rated phase rms voltage'),
        ScaledAliasAttribute('rated_dc_voltage', np.sqrt(3), 'Un', 'V', 'DC bus volatge', groups=['rated_voltage','rated']),
        ScaledAliasAttribute('In_rms', 1.0/np.sqrt(2), 'In', 'A', 'Rated rms phase current (on phase line)'),
        ScaledAliasAttribute('rated_speed_rpm', 30/np.pi, 'vn', 'rpm', 'Rated speed.', groups=['rated_speed','rated']),
        AliasAttribute('rated_current', 'In_rms', groups=['rated_current','rated']),
        AliasAttribute('rated_ac_phase_voltage', 'Un_rms', groups=['rated_voltage','rated']),
        DerivedAttribute('Pn', 'W', 'Rated power', lambda vn, Tn: vn*Tn),
        DerivedAttribute('fn', 'Hz', 'Rated frequency (electrical).', 
                         lambda N, vn: N*vn/(2*np.pi)),    
        AliasAttribute('rated_power', 'Pn', groups=['rated_power', 'rated']),
        AliasAttribute('rated_frequency', 'fn', groups=['rated_speed','rated']),
    ]
    _DERIVATE_RULES = [
        DerivateRule('vn', lambda Pn, Tn: Pn/Tn),
        DerivateRule('vn', lambda N, fn: 2*np.pi*fn/N),
        DerivateRule('Tn', lambda vn, Pn: Pn/vn),
    ]
    _VALIDATORS = [
        Validator(lambda Un_rms, In_rms, Pn: 3*Un_rms*In_rms > Pn, 'Mechanical power output must not be greater then electrical power input')
     ]

    def __init__(self, *args, **kwargs):
        if 'validate' not in kwargs:
            super(MotorPlateParametersRotary, self).__init__(*args, validate = False, **kwargs)
        else:
            super(MotorPlateParametersRotary, self).__init__(*args, **kwargs)

### Модели ротационных двигателей

Собраны из разных источников, указанных в комментариях. Не во всех случаях однозначтн понятно какие варианты параметров используются.

In [ ]:

models = [
    MotorPlateParametersRotary('APM-FEP30AMK3', n_pole_pairs = 4, 
                         rated_ac_voltage = 220, rated_power = 3000, rated_current = 9.94, rated_speed_rpm = 3000, 
                         Rll = 1.3, Lll = 0.0047, J = 19.04e-4,
                         Kemf_llrms_rpm = 0.086 / np.sqrt(2)),
    # 130SPSM22-15220EAM
    MotorPlateParametersRotary('130SPSM22-15220EAM', n_pole_pairs = 5, 
                         rated_ac_voltage = 220, rated_speed_rpm = 2000, rated_current = 8, rated_torque = 7.16,
                         Rll = 0.65, Lll = 0.0047),
    # Parameter Identification of PMSM with Considering Nonlinearity of the Inverter
    # AC voltage from catalog (AC400V, but accoding to charts real rated voltage is AC 200V)
    # incorrect value in Table 2-2 (0.0567), Fm = 0.0225 calculated directly from no-load experiment data
    MotorPlateParametersRotary('TS4614N7124E200', n_pole_pairs = 2,  
                         rated_ac_voltage = 200, rated_torque = 2.39, rated_speed = 3000 * 2*np.pi/60, rated_current = 5.0, 
                         R = 0.43, L = 0.0026, J = 1.3e-4,
                         Fm = 0.225),
    # Implementation Issues of Flux Linkage Estimation on Permanent Magnet Machine Position Sensorless Drive at Low Speed
    # !!! DC bus voltage in specification
    MotorPlateParametersRotary('Implementation Issues of...', n_poles = 8, 
                         rated_ac_voltage = 50, rated_torque = 0.16, rated_speed_rpm = 3000, rated_current = 1.0, 
                         R = 4.7, L = 0.0047,
                         Kemf_rpm = 0.0107), # not clear, V/rpm in specification accorgin to graphs is pahse voltage amplitude to speed in rpm
    # ИДЕНТИФИКАЦИЯ ПАРАМЕТРОВ СИНХРОННЫХ ДВИГАТЕЛЕЙ С ПОСТОЯННЫМИ МАГНИТАМИ НА ОСНОВЕ ИХ ЧАСТОТНОГО АНАЛИЗА
    # Некорректный набор параметров!!! nominal_power > 3 * nominal_voltage_rms * nominal_current_rms
    MotorPlateParametersRotary(name='ИДЕНТИФИКАЦИЯ ПАРА...', n_pole_pairs = 3, 
                         rated_ac_voltage = 110, rated_speed = 314, rated_current = 14.1, rated_power = 5500,
                         R = 0.153, L = 0.0017, J = 0.036,
                         Fm = 0.106,
                         desc = 'Некорректная модель: КПД > 100%'),
    # Control and Parameter Identification of a Permanent Magnet Synchronous Motor with a LC-Filter, pg. 65
    # explicitly pahse voltage! explicitly peak flux linkage
    MotorPlateParametersRotary('Control and Parameter Identification...', n_pole_pairs = 4, 
                         rated_ac_phase_voltage = 165, rated_speed_rpm = 4500, rated_current = 19.5, rated_torque = 20,
                         R = 0.18, L = 0.002, J = 0.0158,
                         Fm = 0.123),
    # Study on the Permanent Magnet Demagnetization Fault in Permanent Magnet Synchronous Machines
    MotorPlateParametersRotary('Study on the Per...', n_pole_pairs = 3, 
                         rated_ac_voltage = 380, rated_speed_rpm = 6000, rated_current = 2.9, rated_torque = 2.3,
                         R = 2.6, L = 0.0096, J = 0.000235,
                         Kemf_llrms_rpm =  0.0576),
    # Robust Diagnosis Method Based on Parameter Estimation for an Interturn Short-Circuit Fault in Multipole PMSM under High-Speed Operation
    # Pn = 2.2 kW, Tn = 7 в дркгой статье
    MotorPlateParametersRotary('FMAIN22-BBFB1', n_pole_pairs = 3, 
                         rated_ac_voltage = 212, rated_speed_rpm = 4500, rated_current = 8.2, rated_torque = 4.7, # 7 in 
                         R = 0.44, L = 0.0006+(0.00309+0.00547)/2,
                         Fm = 0.0976),
    # https://electronics.stackexchange.com/questions/628154/pmsm-motor-full-speed-shorting-over-low-side-fets
    # no info about pole pairs
    MotorPlateParametersRotary('electricalenginering...', desc = 'Число полюсов неизвестно. Взято 4 пары.', 
                         n_pole_pairs = 4, 
                         rated_ac_voltage = 48, rated_speed_rpm = 3000, rated_current = 19.5, rated_torque = 2.39,
                         Rll = 0.066, Lll = 0.000151, J = 1.82e-4, # excplicitly ll
                         Kemf_llrms_rpm = 0.0074, Kt = 0.123 / np.sqrt(2)), # NOTE: it is posible Kt is incorrect in datasheet, because rated_torque was divided by rated_current which is rms
                         #Kemf_llrms_rpm = 0.0074*np.sqrt(2), Kt = 0.123), # NOTE: it is posible Kt is incorrect in datasheet, because rated_torque was divided by rated_current which is rms
                         #Kemf_llrms_rpm = 0.0074), # NOTE: it is posible Kt is incorrect in datasheet, because rated_torque was divided by rated_current which is rms
                         #Kemf_rpm = 0.0074*np.sqrt(2)), # NOTE: it is posible Kt is incorrect in datasheet, because rated_torque was divided by rated_current which is rms
    # https://www.electrocraft.com/files/downloads/Datasheets/bldc/RPX22-DataSheet-US.pdf
    # BLDC.... It seems there are  DC motor parameters
    #MotorPlateParametersRotary('BLDC RPX22-042V48', n_poles = 8, 
    #                     rated_dc_voltage = 48, rated_speed_rpm = 9000, rated_current = 1.3, rated_torque = 0.042,
    #                     R = 9.7, L = 0.0036, J = 1.2 * 0.001 * 0.01**2, # J in g*cm**2
    #                     Kemf_llrms_rpm = 0.0033, Kt = 0.032),
]                       

#### Верификация моделей

In [ ]:
table = []
for model in models:
    try:
        model.validate()
    except Exception as e:
        table.append([model.name, False, str(e)])
    else:
        table.append([model.name, True, ''])
display(tabulate(table, headers=['Model', 'Validation result', 'Reason'], tablefmt='html'))

### Сравниваемые варианты проведения эксперимента определения номинального режима

* `scalar_motor_pos`, `scalar_motor_neg` --- разные ветви решения для режима, соответвующего частотному управлению.
* `mtpa` --- режим полеориентированного управления,
* `bldc` --- режим двигателя постоянного тока.

In [ ]:
from foc_base import PointDQ
from pmsm import flux_from_nominal_motor_mode, flux_from_nominal_generator_mode

def scalar_motor_pos(m):
    return flux_from_nominal_motor_mode(m.R, m.L, m.N, m.vn, m.Pn, m.In, m.Un, sign=1, J = m.J)
def scalar_motor_neg(m):
    return flux_from_nominal_motor_mode(m.R, m.L, m.N, m.vn, m.Pn, m.In, m.Un, sign=-1, J = m.J)
def scalar_generator_pos(m):
    return flux_from_nominal_generator_mode(m.R, m.L, m.N, m.vn, m.Pn, m.In, m.Un, sign=1, J = m.J)
def scalar_generator_neg(m):
    return flux_from_nominal_generator_mode(m.R, m.L, m.N, m.vn, m.Pn, m.In, m.Un, sign=-1, J = m.J)
def mtpa(m):
    Iq = m.In
    Fm = m.Tn / (3/2*m.N*m.In)
    Ud = - m.N*m.vn*m.L*Iq
    Uq = m.R*Iq + m.N*m.vn*Fm
    return Fm, PointDQ(0.0, Iq), PointDQ(Ud, Uq), None
def bldc(m):
    Uq = m.Un
    tgI = m.R/(m.vn*m.N*m.L)
    cosI = np.sqrt(1 / (1 + tgI**2))
    Id = m.In * cosI
    Iq = Id * tgI
    Fm = (Uq - m.R*Iq - m.vn*m.N*m.L*Id) / (m.vn*m.N)
    return Fm, PointDQ(Id, Iq), PointDQ(0.0, Uq), None

flux_nominal = [ scalar_motor_pos, scalar_motor_neg, mtpa, bldc ]

### Результаты сравнения вариантов проведения эксперимента определения номинального режима

В таблице указано соответвие оценки потокосцепления, полученной в преполождении о разных способах проведения эксперимента номинального режима, с "истинной". 

In [ ]:
from tabulate import tabulate
table = []
for m in models:
    Fm_variants = [ func(m)[0] for func in flux_nominal]
    if not np.isnan(m.Fm):
        aFm = np.array(Fm_variants)
        aFm[np.isnan(aFm)] = +np.inf
        best_ind = np.argmin(np.abs(aFm - m.Fm))
        row = [m.name, f'{m.Fm:.4}']
        for v in Fm_variants:
            if not np.isnan(v):
                row.append(f'{v:.4} ({abs(v - m.Fm)/m.Fm*100:3.0f}%)')
            else:
                row.append('<span style="color: red;">N/A</span>')            
        row[best_ind+2] = f'<b>{row[best_ind+2]}</b>'
    else:
        row = [m.name, 'N/A'] + [ f'{v:.4}' for v in Fm_variants]
    row.append(m.desc)
    table.append(row)
    
display(tabulate(table, headers = [ 'model', 'Fm'] + [func.__name__ for func in flux_nominal] + ['Comments'], tablefmt="unsafehtml"))

### Визуализация установившегося режима для разных вариантов проведения эксперимента определения номинального режима

Отображается положежеие вектора тока и и вектора напряжения в подвижной системе координат d-q.


In [ ]:
import matplotlib.pyplot as plt

def visualize_dq(ax: plt.Subplot, I: PointDQ, U: PointDQ, title):
    def symmetrize(ax: plt.Subplot):
        xlim = np.max(np.abs(ax.get_xlim()))
        ylim = np.max(np.abs(ax.get_ylim()))
        ax.set_xlim((-xlim, xlim))
        ax.set_ylim((-ylim, ylim))
    # current
    ax.plot([0, I.d], [0, I.q], label='I', color='red')
    ax.set_xlabel('Id')
    ax.set_ylabel('Iq')
    ax.set_aspect('equal')
    ax.set_title(title)
    symmetrize(ax)
    #ax.grid(True)
    # voltage
    axU = ax.twinx().twiny()
    axU.plot([], [], label='I', color='red')
    axU.plot([0, U.d], [0, U.q], label='U', color='blue')
    axU.set_xlabel('Ud')
    axU.set_ylabel('Uq')
    axU.set_aspect('equal')
    axU.set_title(title)
    axU.legend()
    symmetrize(axU)

m = models[5]
for m in models:
    fig, axes = plt.subplots(1, len(flux_nominal), figsize=(16, 4), constrained_layout=True)
    fig.suptitle(f'{m.name} (Fm = {m.Fm:.4} Вб, {m.desc})')
    for k, method in enumerate(flux_nominal):
        Fm, I, U, stable = method(m)
        if I is not None and U is not None:
            visualize_dq(axes[k], I, U, f'{method.__name__},\n Fm = {Fm:.4},\n stable {stable}')
plt.show()